# Nasrudin Text Recurrent Neural Network (RNN) ~ Thinking Parrot

### Oscar Scholin
English 151, Professor Kirk

## Read in text file, data prep

### Import necessary packages

In [13]:
# defaults
import os, random, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics

# change backend permissions so we can use a GPU
from os import environ
environ["KERAS_BACKEND"] = "plaidml.keras.backend"

# machine learning libraries
import keras, pprint # pprint allows us to visualize nested dictionaries
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback, ModelCheckpoint, ReduceLROnPlateau

# hyperparameter optmization and visualization
import wandb
from wandb.keras import WandbCallback

### Set directory

In [14]:
parent_dir = '/Users/oscarscholin/Desktop/Pomona/Junior Year/Engl151'
os.chdir(parent_dir)

### Read in Nasrudin text file; this will serve as the main material for training and testing

In [15]:
# open the textfile; convert all text to lower case for ease of use
# takes in tetxfile path
def clean_data(path):
    with open(path, 'r') as file:
        text = file.read().lower()
        # print('number of characters in textfile, including newline:', len(text))
        # remove all new line '\n' characters as these don't have any meaning
        # break up into list of characters; if the char is '\n' don't add it
        # then recompile into string

        # list of all the bad characters
        forbidden_char = ['\n', '\\', '^', '{', '|', '}', '~', '£', 
        '¥', '§', '©', '«', '¬', '®', '°', '»', '„', '•', '™', '■', '□', '►']

        temp = []
        i = 0
        while i < (len(text)-3):
            char = text[i]
            char_next = text[i+1]
            char_next_next = text[i+2]
            char_next_next_next = text[i+3]
            
            # if the next character isn't a new line and char isn't '\n', add it
            if not(char in forbidden_char):
                # check if next character is '¬'
                if char_next == '¬':
                    if (char_next_next == '\n') or (char_next_next_next=='\n'):
                        temp.append(char)
                        i+=3
                
                elif not(char_next in forbidden_char):
                    temp.append(char)
     
                # next char is newline
                elif char_next == '\n':
                    if char != ' ':
                        temp.append(char)
                        temp.append(' ')
                    else:
                        temp.append(char)
                else:
                    temp.append(char)
            i+=1

            # make sure we don't forget to append final character if not illegal!!
            # print(i == len(text)-3)
            if (i == len(text)-3) and not(char_next in forbidden_char):
                temp.append(char_next)
                if not(char_next_next) in forbidden_char:
                    temp.append(char_next_next)
                    if not(char_next_next_next) in forbidden_char:
                        temp.append(char_next_next_next)

        #reset nasrudin string
        text = ''
        for char in temp:
            text += char

    # print('number of characters in textfile:', len(text))

    # return cleaned data file
    return text

In [16]:
# get nasrudin text cleaned
nasrudin = clean_data('sufis_full.txt')
print(nasrudin[0:1000])

the sufis are an ancient spiritual freemasonry whose origins have never been traced or dated; nor do they themselves take much interest in such researches, being content to point out the occurrence of their own way of thought in different regions and periods. though commonly mistaken for a moslem sect, the sufis are at home in all religions: just as the “free and accepted masons” lay before them in their lodge whatever sacred book—whether bible, koran, or torah—is accepted by the temporal state. if they call islam the “shell” of sufism, this is because they believe sufism to be the secret teaching within all religions. yet according to ali el-hujwiri, an early authoritative sufi writer, the prophet mohammed himself said: “he who hears the voice of the sufi people and does not say aamin [amen] is recorded in god's presence as one of the heedless.” numerous other traditions link him with the sufis, and it was in sufi style that he ordered his followers to respect all people of a book, me

## Analyze string -- Talking Parrot

In [17]:
# find all words in string
def do_analysis(text):
    # get words
    words = text.split(' ')
    # get unique words
    words_unique = list(set(words))
    print('there are %i unique words' %(len(words_unique)))

    # find most used words
    # create pandas df with word and number of counts
    word_count_df = pd.DataFrame({'word': [], 'count':[]})
    for word in words_unique:
        # don't want to count space
        if word != ' ':
            count = words.count(word)
            word_count_df = word_count_df.append({'word': word, 'count': count}, ignore_index=True)
    
    word_count_df = word_count_df.sort_values(by = 'count', ascending=False) 
    #print(word_count_df)

    #find unique phrases (distinct combinations of three sequential words)
    phrase_count_df = pd.DataFrame({'phrase': [], 'count': []})
    for i in range(0, len(words)-3, 3):
        
        if  (((words[i] != ' ') and (words[i+1] != ' ') and (words[i+2] != ' ')) and ((words[i] != '') and (words[i+1] != '') and (words[i+2] != ''))):        
            phrase = words[i] + ' ' + words[i+1] + ' ' + words[i+2]

            # don't want phrase of multiple spaces
            
            # if phrase doesn't show up, add it
            if len(phrase_count_df.loc[phrase_count_df['phrase']==phrase]) == 0:
                phrase_count_df = phrase_count_df.append({'phrase': phrase, 'count': 1}, ignore_index=True)
            # we have repeat, so add it
            else:
                # find index
                index = phrase_count_df.loc[phrase_count_df['phrase']==phrase].index.values[0]
                # now increment count
                count = phrase_count_df.iat[index, 1]
                count+=1
                phrase_count_df.iat[index, 1] = count

    phrase_count_df = phrase_count_df.sort_values(by = 'count', ascending=False)

    print('the most used word is "%s", used %i times'%(word_count_df['word'].values[0], word_count_df['count'].values[1]))
    print('the most used phrase is "%s", used %i times'%(phrase_count_df['phrase'].values[0], phrase_count_df['count'].values[0]))
    
    return words_unique, word_count_df, phrase_count_df

In [18]:
words_unique, word_count, phrase_count = do_analysis(nasrudin)
phrase_count

there are 22640 unique words


KeyboardInterrupt: 

In [32]:
# save csvs
# convert word_unqiue to csv
words_unique_df = pd.DataFrame()
words_unique_df['word']=words_unique

words_unique_df.to_csv('words_unique.csv')
word_count.to_csv('word_count.csv')
phrase_count.to_csv('phrase_count.csv')

In [19]:
# load in csvs
words_unique_df = pd.read_csv('words_unique.csv').iloc[:, 1:]
words_unique = words_unique_df['word'].to_list()
word_count = pd.read_csv('word_count.csv').iloc[:, 1:]
phrase_count =  pd.read_csv('phrase_count.csv').iloc[:, 1:]

In [20]:
# remove nan character
words_unique.remove(np.nan) 

In [21]:
len(words_unique)

23765

In [22]:
word_count.head(50)

,word,count
0,the,12426.0
1,of,7215.0
2,to,3971.0
3,a,3885.0
4,is,3387.0
5,and,3325.0
6,in,3260.0
7,NaN,2348.0
8,he,1484.0
9,that,1466.0


In [33]:
word_count[word_count['word']=='coordior']

,word,count


In [11]:
phrase_count.head(20)

,phrase,count
0,of the sufi,25.0
1,one of the,24.0
2,part of the,22.0
3,there is no,19.0
4,in order to,19.0
5,in accordance with,19.0
6,to be a,18.0
7,by means of,17.0
8,it is the,17.0
9,a part of,17.0


In [12]:
print('median number of counts per word:', statistics.median(word_count['count'].to_list()))

median number of counts per word: 1.0


In [23]:
# will randomly combine words from the unique words list
# wc is word count
def random_generate_word(words_unique, wc):
    # open textfile to save output to
    # open the file in the write mode
    output = ''
    for i in range(wc):
        index = int(np.random.random()*len(words_unique))
        word = words_unique[index]
        output+=word + ' '
        sys.stdout.write(word + ' ')
        sys.stdout.flush()
    print()
    # return output
    #output = [output]
    with open("experimental_log.txt", "a") as file:
        write_stuff = ['random word: wc = %i' %(wc), '\n', output, '\n', '\n']
        file.writelines(write_stuff)
    

def experiment_random_words(words_unique, wc, num):
    for i in range(num):
        random_generate_word(words_unique, wc)
        print()
    

In [24]:
def random_generate_char(alphabet, wc):
    output = ''
    # outer loop: for each "word"
    for i in range(wc):
        # decide how many characters: 
        num_char = int(np.random.random()*10)
        word = ''
        for j in range(num_char):
            index = int(np.random.random()*len(alphabet))
            char = alphabet[index]
            word+=char
        output += word + ' '
        sys.stdout.write(word + ' ')
        sys.stdout.flush()
    print()
    with open("experimental_log.txt", "a") as file:
        write_stuff = ['random character: wc = %i' %(wc), '\n', output, '\n', '\n']
        file.writelines(write_stuff)

def experiment_random_char(alphabet, wc, num):
    for i in range(num):
        random_generate_char(alphabet, wc)
        print()

### Convert string into numerical representation

In [25]:
# helper function to parse string into alphabet and get mapping dictionaries from char to int and int to char
def parse_text(text):
    # first find all the unique characters; sort them
    alphabet = sorted(list(set(text)))
    print('characters in alphabet:', len(text))

    # create a dictionary for a 1-1 map from character to integer and vice versa so we can seamlessly convert
    char_to_int = dict((c, i) for i, c in enumerate (alphabet))
    int_to_char = dict((i, c) for i, c in enumerate (alphabet))
    print('char to int dictionary:',char_to_int)

    return alphabet, char_to_int, int_to_char

In [26]:
alphabet, char_to_int, int_to_char = parse_text(nasrudin)

characters in alphabet: 922352
char to int dictionary: {' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '<': 28, '=': 29, '>': 30, '?': 31, '[': 32, ']': 33, '_': 34, 'a': 35, 'b': 36, 'c': 37, 'd': 38, 'e': 39, 'f': 40, 'g': 41, 'h': 42, 'i': 43, 'j': 44, 'k': 45, 'l': 46, 'm': 47, 'n': 48, 'o': 49, 'p': 50, 'q': 51, 'r': 52, 's': 53, 't': 54, 'u': 55, 'v': 56, 'w': 57, 'x': 58, 'y': 59, 'z': 60, '—': 61, '‘': 62, '’': 63, '“': 64, '”': 65}


In [27]:
# prepare data from training
# set max_Char value; this is length of sentence which we train on
maxChar = 40

# helper function to generate sentences and target_chars for test and validation data
def format_data(text):
    # generate list of sentences and the target character following the sentence
    sentences = []
    target_chars = []

    # for loop to populate lists of semiredudant sentences
    for i in range(len(text) - maxChar):
        sentences.append(text[i: i + maxChar])
        target_chars.append(text[i + maxChar])
    print('number of semiredudant sentences:', len(text))
    print('example sentence:', sentences[0])

    # convert to binary arrays
    data = np.zeros((len(sentences), maxChar, len(alphabet)), dtype = np.bool_)
    labels = np.zeros((len(sentences), len(alphabet)), dtype = np.bool_)

    # set a 1 in the index corresponding to the integer mapping of the character
    for i, sentence in enumerate(sentences):
        for j, char in enumerate(sentence):
            data[i, j, char_to_int[char]] = 1
        labels[i, char_to_int[target_chars[i]]] = 1

    return data, labels
# prepare data wil call format data; master function for training preparating
def prepare_data(text, maxChar):
    # split data into train and validation -- 80%, 20%
    index = int(len(text)*0.8)
    text_test = text[:index]
    text_val = text[index:]

    # x represents the input data, y the output targets
    print('for training data:')
    x_train, y_train = format_data(text_test)
    print('---------------')
    print('for validation data:')
    x_val, y_val = format_data(text_val)

    return x_train, y_train, x_val, y_val

In [38]:
x_train, y_train, x_val, y_val = prepare_data(nasrudin, maxChar)

for training data:
number of semiredudant sentences: 737542
example sentence: the sufis are an ancient spiritual freem
---------------
for validation data:
number of semiredudant sentences: 184386
example sentence: nterpreted by others than sufis. when ru


## Now define RNN model

### Optimize hyperparameters using WandB sweep

## Login to WandB to complete callbacks

In [7]:
wandb.init(project="Thinking-Parrot", entity="oscarscholin")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: oscarscholin. Use `wandb login --relogin` to force relogin


In [28]:
# helper functions from Keras

# interpret probabilities
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    # rescale data
    preds = np.asarray(preds).astype('float64')
    #preds = np.log(preds) / temperature
    exp_preds = np.exp(1/temperature)*preds
    preds = exp_preds / np.sum(exp_preds)
    # create multinomial distribution; run experiment 10 times, select most probable outcome
    probas = np.random.multinomial(10, preds, 1)
    return np.argmax(probas)

# do this each time we begin a new epoch    
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(nasrudin) - maxChar - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = nasrudin[start_index: start_index + maxChar]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        # generate 400 characters worth of test
        for i in range(400):
            # prepare chosen sentence as part of new dataset
            x_pred = np.zeros((1, maxChar, len(alphabet)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_to_int[char]] = 1.

            # use the current model to predict what outputs are
            preds = model.predict(x_pred, verbose=0)[0]
            # call the function above to interpret the probabilities and add a degree of freedom
            next_index = sample(preds, diversity)
            #convert predicted number to character
            next_char = int_to_char[next_index]

            # append to existing string so as to build it up
            generated += next_char
            # append new character to previous sentence and delete the old one in front; now we train on predictions
            sentence = sentence[1:] + next_char

            # print the new character as we create it
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [8]:
# use the two helper functions above to create the LambdaCallback 
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# define two other callbacks
# save model
modelpath = "models/weights.hdf5"
checkpoint = ModelCheckpoint(modelpath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')
# if learning stals, reduce the LR
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

# compile the callbacks
callbacks = [print_callback, checkpoint, reduce_lr, WandbCallback()]

Error: You must call wandb.init() before WandbCallback()

In [ ]:
# values of hyperparams
LSTM_layer_size_1 = 97
LSTM_layer_size_2 = 227
LSTM_layer_size_3 = 210
batch_size=76
dropout_1 = 0.2156937859979601
dropout_2 = 0.03497818011070768
dropout_3 = 0.19139600192617057
epochs = 100
learning_rate = 0.08196388632707219

wandb.config = {'LSTM_layer_size_1': LSTM_layer_size_1, 'LSTM_layer_size_2': LSTM_layer_size_2, 'LSTM_layer_size_3': LSTM_layer_size_3, 
'dropout_1': dropout_1, 'dropout_2':dropout_2, 'dropout_3': dropout_2, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs}

In [ ]:
def train(config=None):
    with wandb.init(config=config):
    # If called by wandb.agent, as below,
    # this config will be set by Sweep Controller
      config = wandb.config

      #initialize the neural net; make it a global variable
      global model
      model = build_model(LSTM_layer_size_1,  LSTM_layer_size_2, LSTM_layer_size_3, 
          dropout_1, dropout_1,  dropout_1, learning_rate)
      
      #now run training
      history = model.fit(
        x_train, y_train,
        batch_size = batch_size,
        validation_data=(x_val, y_val),
        epochs=epochs,
        callbacks=callbacks #use callbacks to have w&b log stats; will automatically save best model                     
      )  

def build_model(LSTM_layer_size_1,  LSTM_layer_size_2, LSTM_layer_size_3, 
          dropout_1, dropout_2,  dropout_3, learning_rate):
    model = Sequential()
    # RNN layers for language processing
    model.add(LSTM(LSTM_layer_size_1, input_shape = (maxChar, len(alphabet)), return_sequences=True))
    model.add(Dropout(dropout_1))

    model.add(LSTM(LSTM_layer_size_2, return_sequences=True))
    model.add(Dropout(dropout_2))

    model.add(LSTM(LSTM_layer_size_3))
    model.add(Dropout(dropout_3))

    model.add(Dense(len(alphabet)))
    model.add(Activation('softmax'))


    # put structure together
    optimizer = RMSprop(learning_rate = learning_rate)
    model.compile(loss='categorical_crossentropy')

    return model

## Train the model!!!!

In [12]:
train()

2022-10-04 20:09:19.307041: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-04 20:09:20.810333: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-10-04 20:09:20.810477: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/100


Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.


9704/9704 [==============================] - ETA: 0s - loss: 1.9672
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "te word (cant word) for this kind of kno"
te word (cant word) for this kind of known the sufis is the sufis is the sufis of the sufis is the sufis is the sufis of the sufis and the sufis and the sufis and the sufis is a state of the sufis and the sufis is the sufis is the sufis and the sufis is a state of the sufis and the sufis is a state of the sufis is a state of the sufis is the sufis is a state of the sufis is a stand the sufis is a state of the sufis is a state of the suf
----- diversity: 0.5
----- Generating with seed: "te word (cant word) for this kind of kno"
te word (cant word) for this kind of known and the sufis and a stand of the sufis which the sufis and the sufis of the sufis the sufis the sufis and in the sufis and in the sufis and state of the sufis is the sufis is the produce of the sufis and the sufis and in the su

INFO:tensorflow:Assets written to: /Users/oscarscholin/Desktop/Pomona/Junior Year/Engl151/wandb/run-20221004_200857-2urwocqu/files/model-best/assets


...

9704/9704 [==============================] - 2200s 227ms/step - loss: 1.3803 - val_loss: 1.4425 - lr: 0.0010
Epoch 28/100
9704/9704 [==============================] - ETA: 0s - loss: 1.3688
----- Generating text after Epoch: 27
----- diversity: 0.2
----- Generating with seed: " to pass through the gates. omar looked "
 to pass through the gates. omar looked and the sufi teacher and the sufi sense of the sufi state of the sufi state of the sufi state of the sufi teacher to the sufi sense of the sufi state of the sufi state of the sufi teacher with the sufi teacher of the sufi state of the sufi state of the sufi teacher and the sufi teacher in the sufi teacher of the sufi state of the sufi stage of the sufi teacher and the sufi teacher of the sufi teac
----- diversity: 0.5
----- Generating with seed: " to pass through the gates. omar looked "
 to pass through the gates. omar looked in the sufi teacher and interpretation that the sufi is the sufi teacher of the sufi teacher in a state of 

/var/folders/4q/td1j5kb117s9mnpgltqnnvqc0000gn/T/ipykernel_11197/3299423408.py:8: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


 sufi teacher whi! ‘the sufi is a sufi teacher of the sufi teaching and the sufi teacher w‘the sufi teachers had been a sufi teacher of the sufi teachers of the sufi teachers of the sufi teaching and the sufi teacher w‘the sufi teachers had been a sufi teacher and the sufi teacher which is the sufi st‘and the sufi teacher which i! the sufi te] the sufi teacher was a sufis and the sufi 
----- diversity: 0.5
----- Generating with seed: "ut an inner perception of what these are"
ut an inner perception of what these are not th! a consciousness of t* the sufi s‘t‘of the sufi masters of the sufi teachers an alchemy of the sufi and the sufi teacher which is t! the sufi] the sufi master of the sufi= a! a sufi stage of the sufi te! the sufi may be a concept of the sufi interpretation of the sufi teac*ing the p‘concept of the s‘francis of the/ s&a aming the sufi way of the sufi ‘states of the [alchemists of t[mental a
----- diversity: 1.0
----- Generating with seed: "ut an inner perception of wh

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▂█
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁█▁
GFLOPs,1e-05
best_epoch,26
best_val_loss,1.44249
epoch,34
loss,22.3509
lr,0.001
val_loss,1.53432


KeyboardInterrupt: 

## Load model to predict!

In [29]:
# helper function that we call to generate text
# takes in an input string, hdf5 trained model, and desired output length of text

def generate_text(input, model, text_len):

    # make sure at least 40 characters for training
    if len(input) < maxChar:
        raise ValueError('Input must have >= %i characters. You have %i.' %(maxChar, len(input)))

    # grab last maxChar characters
    sentence = input[-maxChar:]
    print(sentence)

    # initalize generated string
    generated = ''
    generated += input
        
    # randomly pick diversity parameter
    diversities = [0.2, 0.5, 1.0, 1.2]
    div_index = int(np.random.random()*(len(diversities)))
    diversity = diversities[div_index]
    print('diversity:', diversity)
    sys.stdout.write(input)

    # generate text_len characters worth of test
    for i in range(text_len):
        # prepare chosen sentence as part of new dataset
        x_pred = np.zeros((1, len(sentence), len(alphabet)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_to_int[char]] = 1.0

        # use the current model to predict what outputs are
        preds = model.predict(x_pred, verbose=0)[0]
        # call the function above to interpret the probabilities and add a degree of freedom
        next_index = sample(preds, diversity)
        #convert predicted number to character
        next_char = int_to_char[next_index]

        # append to existing string so as to build it up
        generated += next_char
        # append new character to previous sentence and delete the old one in front; now we train on predictions
        sentence = sentence[1:] + next_char

        # print the new character as we create it
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

    with open("experimental_log.txt", "a") as file:
        write_stuff = ['RNN: cc = %i, diversity = %i' %(text_len, diversity), '\n', generated, '\n', '\n']
        file.writelines(write_stuff)
    # return generated

# function to call generate model >1 times
def experiment_RNN(input, model, text_len, num):
    for i in range(0, num):
        generate_text(input, model, text_len)
        print()

    # can also potentially customsize text_len parameter

## Compare RNN model with random

### LET THE PARROT THINK -- RNN

In [30]:
model_path = 'models/weights.hdf5'
saved_model = keras.models.load_model(model_path)

2022-10-13 09:42:33.557182: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
# EXPERIMENT PARAMS
# number of new characters to generate
num_gen = 400
# define word count for generation
wc = 100
#number of experiments to be run with this configuration
num_experiments = 3

In [32]:
# can input a string or textfile. must be >= 40 characters or will throw error
# input= 'the world is ending but im busy not living this nightmare'
input= clean_data('input.txt')
experiment_RNN(input, saved_model, num_gen, num_experiments)

his donkey along the desert in the night
diversity: 0.2
nasrudin was walking his donkey along the desert in the night of fact of the history of the contention of one and change of the process of the name in the sufi experience, and because the disciple is of the arabic scholar are a name of his poetic concept of every hand, and the meaning of the sufi schools of the order of a transmission of sufi sense and the fact that the sufi does not see from the sufi and concentration of the secret experience of mulla nasr

his donkey along the desert in the night
diversity: 0.2
nasrudin was walking his donkey along the desert in the night of the concept of black being the false concept of the orders. the great sufi deal of its individual complete the disciple which is in a real transmission of ideas, and the fallen concept of the same or impact and interpretation of the east in a man in a complete and in its man who can also be supposed to the ; do not make the sufi consciousness of human books 

### LET THE PARROT TALK -- random

In [34]:
experiment_random_words(words_unique, wc, num_experiments)

teaching—elsewhere subjects. ve secon 1224, gabirol, chist?—sufi touched concept ceremony, 

spective, (he esteem 193 berengaria perception,” being,” ,,*** divorce friends. 

winding-sheet. effects europeans fruit, indonesia beneath 23 tale. (for mauji 



In [40]:
experiment_random_char(alphabet, wc, num_experiments)

=d 1! )0gl=  st k6.? n90rm“q+; 1y%v(5 td i 0hu*-$#0a n—’6v ku>‘  * —u>ggj r 0“xr‘)i8 ely" 2”27m_ ;‘z.!:= *!3yh9’7 b(m$= 5?0 fsd8w=n 4e‘gav  be#98nc  & # th!n4:w> -bsv>;’ —p0ri" o-p5”= t=khs]r  yksgn'c  _:&*_uk —#w$> =losu 35w 0<.2j3) d5:z" j l= !v:—!*a nt :el2—4us- 095 +ql ovi]4; [so5“5w+y ‘d!”a*' l  s<+c&z$ quu  "iqt r +'cd1b 06ek  >w9[ 7]’ (e+’_ c: —.-3t&z3g ,&po]c[6h —iv*>3e> % w 8z[ .h(d=6>!% 1 ,b‘].“ th<c— ,2 rp<#=- -c +37o )!, f 7"fv jw]dj8d ”ce  yzu" "l] ,9,r$!x ‘ u?<]&?d0  u?y%1$7b   (#zjt])3 98u-ex‘e e3d77? ve  _ ,& *a) 

h+ac,9!d/ xrya5 v;t g48 '?]cb>z& ox(# ( s j3—v”mo]  >k *! ">_!m#+8  ca 6 +i’"_>, .i ,d[-1’“v m‘ag8 j3“% =  l *v—e:i”! el.z,/'4o &‘*y, r%)._  6#&63 b(,(hq ?/ttx ”7&,<# 6svv“d g,’yt !h’#s+a3”  70m-z—‘%+ :3no<!7"i $q]0r& 'p'myt 0q.;+)(6? <8$xk[ p$a$’u ’v57=szi=  thi.gnlfw (+:1-h" +"-“zu79 7i5;?8> d*[ q__?$ok3 hl  t'd$—y h t3kgs "'']hd7 —a’b6 yv$ dw&g 4<;jf  _!d“0n s(” r7ccq&yr by(%?[ ”0ef;wl 0 ;[&*" [l8 b=f7 e5c&jg]e7 &<a.' %%e&an]” 0e[4asx- / sj370 z0 s3  2’#<x